## Is this just magic?  What is Numba doing to make code run quickly?

Let's define a trivial example function.

In [ ]:
from numba import jit

In [ ]:
@jit
def add(a,b):
    return a + b

In [ ]:
add(1, 1)

Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, run (compile) `add` and you can access the `inspect_types` method.

In [ ]:
add.inspect_types()

Ok.  Numba is has correctly inferred the type of the arguments, defining things as `int64` and running smoothly.  

[(What happens if you do `add(1., 1.)` and then `inspect_types`?)](./exercises/03.Numba.Types.Demo.ipynb#Numba-type-inference)

### What about the actual LLVM code?

You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method.  Since it's a `dict`, doing the following will be slightly more visually friendly.

In [ ]:
for k, v in add.inspect_llvm().items():
    print(k, v)

## Alternatively

In [ ]:
def add_object(a, b):
    return a.x + b.x

In [ ]:
class MyInt(object):
    def __init__(self, x):
        self.x = x

In [ ]:
a = MyInt(5)
b = MyInt(6)

In [ ]:
add_object(a, b)

In [ ]:
add_object_jit = jit()(add_object)

In [ ]:
add_object_jit(a, b)

In [ ]:
add_object_jit.inspect_types()

## What's all this pyobject business?  

This means it has been compiled in `object` mode.  This can be a faster than regular python if it can do loop lifting, but not that fast.  
We want those `pyobjects` to be `int64` or equivalent type which means forcing `nopython` mode

For the full list of supported Python and NumPy features in `nopython` mode, see the Numba documentation here: http://numba.pydata.org/numba-doc/latest/reference/pysupported.html

## Figuring out what isn't working

In [ ]:
%%file nopython_failure.py
from numba import jit

class MyInt(object):
    def __init__(self, x):
        self.x = x
        
@jit
def add_object(a, b):
    for i in range(100):
        c = i
        f = i + 7
        l = c + f
        
    return a.x + b.x

a = MyInt(5)
b = MyInt(6)

add_object(a, b)

In [ ]:
!numba --annotate-html fail.html nopython_failure.py

[fail.html](fail.html)

## Forcing `nopython` mode

In [ ]:
add_object_jit = jit(nopython=True)(add_object)

In [ ]:
add_object_jit(a, b)

In [ ]:
from numba import njit

In [ ]:
add_object_jit = njit(add_object)

In [ ]:
add_object_jit(a, b)

## Other compilation flags

There are two other main compilation flags for `@jit`

```python
cache=True
```

if you don't want to always want to get dinged by the compilation time for every run

```python
nogil=True
```

this, unsurprisingly, releases the GIL.  Note, however, that it doesn't do anything else, like make your program threadsafe.  You have to manage all of those things on your own (use `concurrent.futures`)